In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS retail_chain_catalog.gold_schema;

CREATE TABLE IF NOT EXISTS retail_chain_catalog.gold_schema.gold_customer_360
USING DELTA
AS
WITH accounts_metrics AS
(
  SELECT
      CustomerID,
      COUNT(AccountId) as total_active_accounts,
      SUM(Balance) as total_current_balance
   FROM retail_chain_catalog.silver_schema.silver_accounts_final_dedupe
   WHERE AccountStatusID = 1
   GROUP BY CustomerID
),loan_metrics AS
(
  SELECT
      AccountID,
      SUM(PrincipalAmount) AS total_outstanding_loan_principal
  FROM retail_chain_catalog.silver_schema.silver_loans_final_dedupe
  WHERE LoanStatusID = 1
  GROUP BY AccountID
),loan_Customer_map AS
(
  SELECT
      acc.CustomerID,
      SUM(total_outstanding_loan_principal) AS total_loan_exposure
      FROM retail_chain_catalog.silver_schema.silver_accounts_final_dedupe acc LEFT JOIN loan_metrics lm ON acc.AccountID = lm.AccountID
      GROUP BY acc.CustomerID
)
SELECT 
    -- 1. Identity & Demographics
    c.CustomerID,
    c.FirstName,
    c.LastName,
    c.DateOfBirth,
    FLOOR(DATEDIFF(CURRENT_DATE(), c.DateOfBirth) / 365.25) AS customer_age,
    ct.TypeName,
  
    -- 2. Concatenated Address
    CONCAT(a.Street, ', ', a.City, ', ', a.Country) AS full_address,
    
    -- 3. Financial Metrics (Coalesce used to handle customers with no accounts/loans)
    COALESCE(am.total_active_accounts, 0) AS total_active_accounts,
    COALESCE(am.total_current_balance, 0.00) AS total_current_balance,
    COALESCE(lcm.total_loan_exposure, 0.00) AS total_loan_exposure,
    
    -- 4. Derived Business Value: Net Position
    (COALESCE(am.total_current_balance, 0) - COALESCE(lcm.total_loan_exposure, 0)) AS net_position,
    
    CURRENT_TIMESTAMP() AS _processed_at

FROM retail_chain_catalog.silver_schema.silver_customers_scdtype2 c
-- Join only current versions of customers
LEFT JOIN retail_chain_catalog.silver_schema.silver_address_cleaned a ON c.AddressID = a.AddressID
LEFT JOIN retail_chain_catalog.bronze_schema.customer_types ct ON c.CustomerTypeID = ct.CustomerTypeID
LEFT JOIN accounts_metrics am ON c.CustomerID = am.CustomerID
LEFT JOIN loan_customer_map lcm ON c.CustomerID = lcm.CustomerID
WHERE c.__END_AT IS NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE retail_chain_catalog.gold_schema.gold_branch_performance 
USING DELTA
AS
WITH daily_transaction_summary AS (
    -- Step 1: Aggregate transactions by Branch, Month, and Type
    SELECT 
        BranchID,
        TRUNC(TransactionDate, 'MM') AS transaction_month,
        TransactionTypeID,
        SUM(Amount) AS total_volume,
        COUNT(TransactionID) AS transaction_count,
        COUNT(DISTINCT AccountOriginID) AS unique_accounts_served
    FROM retail_chain_catalog.silver_schema.silver_transactions_final_dedupe
    GROUP BY 1, 2, 3
),
branch_customer_count AS (
    -- Step 2: Get total unique customers per branch per month
    -- We join with accounts to find the CustomerID associated with the transaction
    SELECT 
        t.BranchID,
        TRUNC(t.TransactionDate, 'MM') AS transaction_month,
        COUNT(DISTINCT a.CustomerID) AS total_unique_customers
    FROM retail_chain_catalog.silver_schema.silver_transactions_final_dedupe t
    JOIN retail_chain_catalog.silver_schema.silver_accounts_final_dedupe a 
      ON t.AccountOriginID = a.AccountID
    GROUP BY 1, 2
)

SELECT 
    -- 1. Branch & Location Details
    b.BranchID,
    b.BranchName,
    addr.City,
    addr.Country,
    
    -- 2. Time Dimension
    dts.transaction_month,
    
    -- 3. Performance Metrics
    tt.TypeName,
    dts.total_volume,
    dts.transaction_count,
    
    -- 4. Reach Metrics (using the specific month's customer count)
    bcc.total_unique_customers,
    
    CURRENT_TIMESTAMP() AS _last_updated_at

FROM daily_transaction_summary dts
JOIN retail_chain_catalog.silver_schema.silver_branches_cleaned b ON dts.BranchID = b.BranchID
LEFT JOIN retail_chain_catalog.silver_schema.silver_address_cleaned addr ON b.AddressID = addr.AddressID
LEFT JOIN retail_chain_catalog.bronze_schema.transaction_types tt ON dts.TransactionTypeID = tt.TransactionTypeID
LEFT JOIN branch_customer_count bcc ON dts.BranchID = bcc.BranchID 
      AND dts.transaction_month = bcc.transaction_month;

In [0]:
%sql
CREATE OR REPLACE TABLE retail_chain_catalog.gold_schema.gold_loan_risk_exposure 
USING DELTA
AS
WITH customer_deposits AS (
    -- Step 1: Calculate total deposits per customer across all their accounts
    SELECT 
        CustomerID,
        SUM(Balance) AS total_deposit_balance
    FROM retail_chain_catalog.silver_schema.silver_accounts_final_dedupe
    GROUP BY CustomerID
),
loan_details AS (
    -- Step 2: Prepare loan metrics and calculate time remaining
    SELECT 
        l.LoanID,
        acc.CustomerID,
        l.PrincipalAmount,
        l.InterestRate,
        ls.StatusName,
        l.EstimatedEndDate,
        -- Calculate days remaining (0 if already passed)
        GREATEST(DATEDIFF(l.EstimatedEndDate, CURRENT_DATE()), 0) AS days_until_maturity
    FROM retail_chain_catalog.silver_schema.silver_loans_final_dedupe l
    JOIN retail_chain_catalog.bronze_schema.loan_statuses ls ON l.LoanStatusID = ls.LoanStatusID
    JOIN retail_chain_catalog.silver_schema.silver_accounts_final_dedupe acc ON l.AccountID = acc.AccountID
)

SELECT 
    -- 1. Identity & Context
    ld.LoanID,
    c.FirstName,
    c.LastName,
    ct.TypeName,
    
    -- 2. Risk Metrics
    ld.StatusName,
    ld.PrincipalAmount,
    ld.InterestRate,
    ld.days_until_maturity,
    
    -- 3. Liquidity Metrics (Loan-to-Deposit Ratio)
    COALESCE(cd.total_deposit_balance, 0) AS total_customer_deposits,
    CASE 
        WHEN COALESCE(cd.total_deposit_balance, 0) > 0 
        THEN (ld.PrincipalAmount / cd.total_deposit_balance) 
        ELSE NULL -- Ratio is undefined if there are no deposits
    END AS loan_to_deposit_ratio,
    
    -- 4. Risk Flagging
    CASE 
        WHEN ld.StatusName = 'Overdue' THEN 'Critical: Immediate Action'
        WHEN ld.StatusName = 'Active' AND (ld.PrincipalAmount / cd.total_deposit_balance) > 1.2 THEN 'Warning: Over-Leveraged'
        WHEN ld.StatusName = 'Paidoff' THEN 'Low Risk: Completed'
        ELSE 'Standard'
    END AS risk_rating,

    CURRENT_TIMESTAMP() AS _report_generated_at

FROM loan_details ld
JOIN retail_chain_catalog.silver_schema.silver_customers_cleaned c ON ld.CustomerID = c.CustomerID
LEFT JOIN retail_chain_catalog.bronze_schema.customer_types ct ON c.CustomerTypeID = ct.CustomerTypeID
LEFT JOIN customer_deposits cd ON ld.CustomerID = cd.CustomerID;

In [0]:
%sql
CREATE OR REPLACE TABLE retail_chain_catalog.gold_schema.gold_transaction_anomalies 
USING DELTA
AS
WITH account_rolling_stats AS (
    -- Step 1: Calculate the rolling 30-day average for each account
    -- This creates a baseline of "normal" behavior
    SELECT 
        TransactionID,
        AccountOriginID,
        TransactionDate,
        Amount,
        TransactionTypeID,
        AVG(Amount) OVER (
            PARTITION BY AccountOriginID 
            ORDER BY CAST(TransactionDate AS LONG) 
            RANGE BETWEEN 2592000 PRECEDING AND CURRENT ROW -- 30 days in seconds
        ) AS avg_30day_spend
    FROM retail_chain_catalog.silver_schema.silver_transactions_final_dedupe
),
daily_cash_flow AS (
    -- Step 2: Calculate total daily inflow vs outflow
    -- Assuming Type 1 is Deposit (Inflow) and Type 2 is Withdrawal (Outflow)
    SELECT 
        DATE(TransactionDate) AS trans_date,
        SUM(CASE WHEN TransactionTypeID = 1 THEN Amount ELSE 0 END) AS total_inflow,
        SUM(CASE WHEN TransactionTypeID = 2 THEN Amount ELSE 0 END) AS total_outflow
    FROM retail_chain_catalog.silver_schema.silver_transactions_final_dedupe
    GROUP BY 1
)

SELECT 
    -- 1. Transaction Details
    ars.TransactionID,
    ars.AccountOriginID,
    a.CustomerID,
    tt.TypeName,
    ars.TransactionDate,
    ars.Amount,
    
    -- 2. Anomaly Detection Logic
    ROUND(ars.avg_30day_spend, 2) AS account_30day_avg,
    CASE 
        WHEN ars.Amount > (ars.avg_30day_spend * 3) THEN 'High Spender Spike'
        WHEN ars.Amount > 10000 THEN 'Large Fixed Threshold Alert'
        ELSE 'Normal'
    END AS anomaly_flag,
    
    -- 3. Daily Context
    dcf.total_inflow AS global_daily_inflow,
    dcf.total_outflow AS global_daily_outflow,
    
    CURRENT_TIMESTAMP() AS _flagged_at

FROM account_rolling_stats ars
JOIN retail_chain_catalog.silver_schema.silver_accounts_final_dedupe a ON ars.AccountOriginID = a.AccountID
JOIN retail_chain_catalog.bronze_schema.transaction_types tt ON ars.TransactionTypeID = tt.TransactionTypeID
LEFT JOIN daily_cash_flow dcf ON DATE(ars.TransactionDate) = dcf.trans_date;